<a href="https://colab.research.google.com/github/Nikita018/Semantic-Segmentation-Suite/blob/master/InceptionDeepNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

def build_inception_deepNet(num_classes):
  local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

  pre_trained_model = InceptionV3(input_shape = (720, 960, 3), 
                                  include_top = False, 
                                  weights = None)

  pre_trained_model.load_weights(local_weights_file)

  for layer in pre_trained_model.layers:
    layer.trainable = False
    
  pre_trained_model.summary()

  last_layer = pre_trained_model.get_layer('mixed7')
  print('last layer output shape: ', last_layer.output_shape)
  last_output = last_layer.output
  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(1024, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x)                  
  # Add a final sigmoid layer for classification
  x = layers.Dense  (num_classes, activation=None)(x)           
  #net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None, scope='logits')
  #model = Model( pre_trained_model.input, x) 

  return x

--2020-03-19 17:00:10--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 2607:f8b0:400c:c14::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  95.9MB/s    in 0.9s    

2020-03-19 17:00:11 (95.9 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [5]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (720, 960, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 720, 960, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 359, 479, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 359, 479, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 359, 479, 32) 0           batch_normalization[0][0]        
______

In [20]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
print(last_output)
inputs = last_output
feature_map_size = tf.shape(last_output)
depth=256
label_size = tf.shape(inputs)[1:3]
num_classes = 32

# Global average pooling
image_features = tf.reduce_mean(inputs, [1, 2], keep_dims=True)
print(image_features)


# image_features = tf.keras.layers.Conv2D(
#     depth, [1, 1], strides=(1, 1), activity_regularizer=None
# )(image_features)
image_features = slim.conv2d(image_features, depth, [1, 1], activation_fn=None)
image_features = tf.image.resize_bilinear(image_features, (feature_map_size[1], feature_map_size[2]))

atrous_pool_block_1 = slim.conv2d(inputs, depth, [1, 1], activation_fn=None)

atrous_pool_block_6 = slim.conv2d(inputs, depth, [3, 3], rate=6, activation_fn=None)

atrous_pool_block_12 = slim.conv2d(inputs, depth, [3, 3], rate=12, activation_fn=None)

atrous_pool_block_18 = slim.conv2d(inputs, depth, [3, 3], rate=18, activation_fn=None)

net = tf.concat((image_features, atrous_pool_block_1, atrous_pool_block_6, atrous_pool_block_12, atrous_pool_block_18), axis=3)
net = slim.conv2d(net, depth, [1, 1], scope="conv_1x1_output", activation_fn=None,reuse=True)
net = tf.image.resize_bilinear(net, size=label_size)
net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None, scope='logits')

last layer output shape:  (None, 43, 58, 768)
Tensor("mixed7/concat:0", shape=(?, 43, 58, 768), dtype=float32)


In [0]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
print(last_output)
inputs = last_output
feature_map_size = tf.shape(last_output)
depth=256
label_size = tf.shape(inputs)[1:3]
num_classes = 32

# Global average pooling
image_features = tf.reduce_mean(inputs, [1, 2], keep_dims=True)
print(image_features)


image_features = tf.keras.layers.Conv2D(
    depth, [1, 1], strides=(1, 1), activity_regularizer=None
)(image_features)
image_features = tf.image.resize_bilinear(image_features, (feature_map_size[1], feature_map_size[2]))

atrous_pool_block_1 = tf.keras.layers.Conv2D(depth, [1, 1], strides=(1, 1), activity_regularizer=None)(inputs)

atrous_pool_block_6 = tf.keras.layers.Conv2D(depth, [3, 3], padding = 'same', dilation_rate = [6,6],activity_regularizer=None)(inputs)

atrous_pool_block_12 = tf.keras.layers.Conv2D(depth, [3, 3], padding = 'same', dilation_rate = [12,12],activity_regularizer=None)(inputs)

atrous_pool_block_18 = tf.keras.layers.Conv2D(depth, [3, 3], padding = 'same', dilation_rate = [18,18],activity_regularizer=None)(inputs)

net = tf.concat((image_features, atrous_pool_block_1, atrous_pool_block_6, atrous_pool_block_12, atrous_pool_block_18), axis=3)
net = tf.keras.layers.Conv2D(depth, [1, 1], activity_regularizer=None)(net)
net = tf.image.resize_bilinear(net, size=label_size)
net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None, scope='logits')

In [70]:
# Keras.conv2D experimentation
# Global average pooling
label_size = tf.shape(inputs)[1:3]
print("input to reduce mean:",inputs)
image_features = tf.reduce_mean(inputs, [1, 2], keep_dims=True)

image_features = tf.keras.layers.Conv2D(depth, [1, 1], strides=(1, 1), activity_regularizer=None)(image_features)
print("Image Features :",image_features)
image_features = tf.image.resize_bilinear(image_features, (feature_map_size[1], feature_map_size[2]))
print("Image Features after Bilinear :",image_features)
atrous_pool_block_1 = tf.keras.layers.Conv2D(depth, [1, 1], activity_regularizer=None)(inputs)
print("block1 :",atrous_pool_block_1)
atrous_pool_block_6 = tf.keras.layers.Conv2D(depth, [3, 3], padding = 'same', dilation_rate = [6,6],activity_regularizer=None)(inputs)
print("block6 :",atrous_pool_block_6)
atrous_pool_block_12 = tf.keras.layers.Conv2D(depth, [3, 3], padding = 'same', dilation_rate = [12,12],activity_regularizer=None)(inputs)
print("block12 :",atrous_pool_block_12)
atrous_pool_block_18 = tf.keras.layers.Conv2D(depth, [3, 3], padding = 'same', dilation_rate = [18,18],activity_regularizer=None)(inputs)
print("block18 :",atrous_pool_block_18)
net = tf.concat((image_features, atrous_pool_block_1, atrous_pool_block_6, atrous_pool_block_12, atrous_pool_block_18), axis=3)
print(net)
net = tf.keras.layers.Conv2D(depth, [1, 1], activity_regularizer=None)(net)
print(net)
net = tf.image.resize_bilinear(net, size=label_size)
print(net)
net = tf.keras.layers.Conv2D(num_classes, [1, 1], activity_regularizer=None)(net)
model = Model( pre_trained_model.input, net)
model.summary() 


input to reduce mean: Tensor("mixed7/concat:0", shape=(?, 43, 58, 768), dtype=float32)
Image Features : Tensor("conv2d_180/BiasAdd:0", shape=(?, 1, 1, 256), dtype=float32)
Image Features after Bilinear : Tensor("ResizeBilinear_39:0", shape=(?, ?, ?, 256), dtype=float32)
block1 : Tensor("conv2d_181/BiasAdd:0", shape=(?, 43, 58, 256), dtype=float32)
block6 : Tensor("conv2d_182/BiasAdd:0", shape=(?, 43, 58, 256), dtype=float32)
block12 : Tensor("conv2d_183/BiasAdd:0", shape=(?, 43, 58, 256), dtype=float32)
block18 : Tensor("conv2d_184/BiasAdd:0", shape=(?, 43, 58, 256), dtype=float32)
Tensor("concat_24:0", shape=(?, 43, 58, 1280), dtype=float32)
Tensor("conv2d_185/BiasAdd:0", shape=(?, 43, 58, 256), dtype=float32)
Tensor("ResizeBilinear_40:0", shape=(?, ?, ?, 256), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     

In [67]:
# slim.conv2D experimentation
image_features = tf.reduce_mean(inputs, [1, 2], keep_dims=True)

image_features = slim.conv2d(image_features, depth, [1, 1], activation_fn=None)
image_features = tf.image.resize_bilinear(image_features, (feature_map_size[1], feature_map_size[2]))

atrous_pool_block_1 = slim.conv2d(inputs, depth, [1, 1], activation_fn=None)

atrous_pool_block_6 = slim.conv2d(inputs, depth, [3, 3], rate=6, activation_fn=None)

atrous_pool_block_12 = slim.conv2d(inputs, depth, [3, 3], rate=12, activation_fn=None)

atrous_pool_block_18 = slim.conv2d(inputs, depth, [3, 3], rate=18, activation_fn=None)

net = tf.concat((image_features, atrous_pool_block_1, atrous_pool_block_6, atrous_pool_block_12, atrous_pool_block_18), axis=3)
net = slim.conv2d(net, depth, [1, 1], scope="conv_1x1_output", reuse=True,activation_fn=None)
net = tf.image.resize_bilinear(net, size=label_size)
print(net)
net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None,reuse=True, scope='logits')

print(net)

Tensor("ResizeBilinear_34:0", shape=(?, ?, ?, 256), dtype=float32)
Tensor("logits_3/BiasAdd:0", shape=(?, ?, ?, 32), dtype=float32)


In [24]:
x = tf.constant([[1., 1.], [2., 2.]])
tf.reduce_mean(x) 

<tf.Tensor 'Mean_12:0' shape=() dtype=float32>

In [0]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
